In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import librosa
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split

#### Extracting data from csv files into numpy arrays

In [ ]:
#extracting data from csv files into numpy arrays
from numpy import genfromtxt
files_data=np.array(['fold_data_1.csv','fold_data_2.csv','fold_data_3.csv','fold_data_4.csv','fold_data_5.csv',
                'fold_data_6.csv','fold_data_7.csv','fold_data_8.csv','fold_data_9.csv','fold_data_10.csv'])
X_train_data= np.array([genfromtxt(file,delimiter=',') for file in files_data])

files_label=(['fold_labels_1.csv','fold_labels_2.csv','fold_labels_3.csv','fold_labels_4.csv','fold_labels_5.csv',
                'fold_labels_6.csv','fold_labels_7.csv','fold_labels_8.csv','fold_labels_9.csv','fold_labels_10.csv'])
Y_train_labels= np.array([genfromtxt(file,delimiter=',') for file in files_label])


In [ ]:
print(np.shape(X_train_data[0]))

#### Converting to Hot Encoder

In [ ]:
Y_train=([to_categorical(Y_train_labels, num_classes=10) for Y_train_labels in Y_train_labels])

#### Reshaping to 1D

In [ ]:
X_train_1D=([np.reshape(X_train_data,(X_train_data.shape[0], 240, 1)) for X_train_data in X_train_data])

#### Reshaping to 2D

In [ ]:
X_train_2D=([np.reshape(X_train_1D,(X_train_1D.shape[0], 60, 4)) for X_train_1D in X_train_1D])
print(np.shape(X_train_2D[0]))

#### CNN formatting

In [ ]:
X_train=([np.reshape(X_train_2D,(X_train_2D.shape[0],60,4,1)) for X_train_2D in X_train_2D])

#### Shapes of the data


In [ ]:
for i in range((10)):
    print("X_data",i+1,"=",np.shape(X_train[i]))
    print("Y_data",i+1,"=",np.shape(Y_train[i]))

#### Model

In [ ]:
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Dropout,merge

In [ ]:
# Model from GitHub

#adding layers and forming the model

#forming model
model=Sequential()

model.add(Conv2D(64,kernel_size=5,strides=1,padding="Same",activation="relu",input_shape=(60,4,1)))
model.add(MaxPooling2D(padding="same"))

model.add(Conv2D(128,kernel_size=5,strides=1,padding="same",activation="relu"))
model.add(MaxPooling2D(padding="same"))
model.add(Dropout(0.3))

model.add(Flatten())

model.add(Dense(256,activation="relu"))
model.add(Dropout(0.3))

model.add(Dense(512,activation="relu"))
model.add(Dropout(0.3))

model.add(Dense(10,activation="softmax"))

#compiling
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [ ]:
import numpy as np
V=np.random.choice([0,1,2,3,4,5,6,7,8,9], 10, replace=False)
T=np.random.choice([0,1,2,3,4,5,6,7,8,9], 10, replace=False)
print("initial V",V)
print("initial T",T)
for i in range(10):
    if V[i]==T[i]:
        print("ERROR\n-------\n------\n-------")
        V=np.random.choice([0,1,2,3,4,5,6,7,8,9], 10, replace=False)
        T=np.random.choice([0,1,2,3,4,5,6,7,8,9], 10, replace=False)

Val=[(V) for V in V]
Test=[(T) for T in T]
print(Val)
print(Test)

for i in range(10):
    
    F=np.array([0,1,2,3,4,5,6,7,8,9])
    
    F = F[F != Val[i]]
    F = F[F != Test[i]]
    print("\nFOLD",i+1,"\n- - - - - - - - - - - - - - - - - - - - ")
    print("Validating of Fold:",Val[i])
    print("Testing of Fold:",Test[i])
    
    Training_set=[F for F in F]
    print("Training off of Folds:",Training_set)
    x_train = np.vstack([X_train[Training_set[0]],X_train[Training_set[1]],X_train[Training_set[2]],X_train[Training_set[3]]
                        ,X_train[Training_set[4]],X_train[Training_set[5]],X_train[Training_set[6]],X_train[Training_set[7]]])
    y_train = np.vstack([Y_train[Training_set[0]],Y_train[Training_set[1]],Y_train[Training_set[2]],Y_train[Training_set[3]]
                        ,Y_train[Training_set[4]],Y_train[Training_set[5]],Y_train[Training_set[6]],Y_train[Training_set[7]]])
   
    print(x_train.shape)
    print(y_train.shape)
    model_history = model.fit(x_train,y_train,batch_size=50,epochs=30,validation_data=(X_train[Val[i]],Y_train[Val[i]]))
    Score=model.evaluate(X_train[Test[i]], Y_train[Test[i]])
    print("Accuracy of Fold",i+1,"=",Score[1])